In [1]:

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
page_num = [1,51]
soup=[]
for pagenums in page_num:
    page = requests.get(f'https://www.imdb.com/search/title/?groups=top_100&sort=user_rating,desc&start={page_num}&ref_=adv_nxt')
    print(page)
    soup.append(BeautifulSoup(page.content,'html.parser'))

<Response [200]>
<Response [200]>


In [8]:
##movie_title and year of release
titles=[]
years= []
descriptions =[]
durations =[]
ratings =[]
votes = []
genres= []
stars = []
directors = []
for item in soup:
    movie_title = item.find_all('h3',class_="lister-item-header") #get the tag containing the movie title and year of release
    movie_discrip = item.find_all('div',class_="lister-item-content") #get tags containing the description


    #Movies and Years
    for movies in movie_title:
        titles.append(movies.text.replace("\n","").split(".")[1].split("(")[0])
        years.append( pd.Series(movies.text.replace('\n','')).str.extract(r'\((\d+)').squeeze()  )
    
      
    #Description
    for discrip in movie_discrip:
        ptags=discrip.find_all('p',class_="text-muted")
        discrips=ptags[1].text.replace('\n','')
        descriptions.append(discrips)
    #Duration
        duration =ptags[0].text.replace('\n','')
        durations.append( pd.Series(duration).str.extract(r'(\d+) min').squeeze())

    #Ratings
    rate = item.select(".lister-item-content")   
    for rates in rate:
        ratings.append( pd.Series(rates.text.replace('\n','')).str.extract(r'(\d+\.\d+)Rate').squeeze()  )
    #Votes
        votes.append( pd.Series(rates.text.replace('\n','')).str.extract(r'Votes\:(\d+,\d+)') .squeeze() )
        

    #Genres
    movie_genre = item.find_all('span',class_="genre")
    for genre in movie_genre:
        genres.append(genre.text.replace("\n","").strip('    '))

    #Stars
    movie_star = item.find_all('p',class_='')
    for star in movie_star:
        stars.append(star.text.replace("\n",'').split('Stars:')[1])
    #Directors
        directors.append(star.text.replace("\n",'').split('|')[0].split(':')[1])

        



years = [int(toint) for toint in years]


In [4]:


start_date = []
end_date = []
for item in soup:
    
    result = item.find_all(class_ = "lister-item-header")
    for tag in result:
        url_codes = tag.a['href']
        url = 'https://www.imdb.com{}locations?ref_=ttfc_ql_5'.format(url_codes)
        response = requests.get(url)
        page = BeautifulSoup(response.content, 'html.parser')
        filming_dates = page.select('.listo')
        
        for tag in filming_dates:
            start_date.append( pd.Series(tag.text.replace("\n", '')).str.extract(r'[fF]ilming [dD]ates\s+(\d+\s\w+\s\d+)').squeeze() )
            end_date.append( pd.Series(tag.text.replace("\n", '')).str.extract(r'\d+\s-\s(\d+\s\w+\s\d+)').squeeze() )

In [5]:
final_data =pd.DataFrame({'Title':titles, 'Descriptions':descriptions,'Release':years,'Filming_start_date':start_date,'Filming_end_date':end_date ,'directors':directors,'rating':ratings,'durations':durations,'votes':votes,'genres':genres,'stars':stars})
final_data.to_csv('IMDB_100best.csv', index=False)


In [6]:
final_data.head(30)

,Title,Descriptions,Release,Filming_start_date,Filming_end_date,directors,rating,durations,votes,genres,stars
0,Skazani na Shawshank,Two imprisoned men bond over a number of years...,1994,16 June 1993,10 September 1993,Frank Darabont,9.3,142,"2,596",Drama,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi..."
1,Ojciec chrzestny,The aging patriarch of an organized crime dyna...,1972,29 March 1971,6 August 1971,Francis Ford Coppola,9.2,175,"1,792","Crime, Drama","Marlon Brando, Al Pacino, James Caan, Diane Ke..."
2,Mroczny Rycerz,When the menace known as the Joker wreaks havo...,2008,18 April 2007,11 November 2007,Christopher Nolan,9.0,152,"2,568","Action, Crime, Drama","Christian Bale, Heath Ledger, Aaron Eckhart, M..."
3,Władca Pierścieni: Powrót króla,Gandalf and Aragorn lead the World of Men agai...,2003,11 October 1999,22 December 2000,Peter Jackson,9.0,201,"1,783","Action, Adventure, Drama","Elijah Wood, Viggo Mortensen, Ian McKellen, Or..."
4,Lista Schindlera,"In German-occupied Poland during World War II,...",1993,1 March 1993,11 May 1993,Steven Spielberg,9.0,195,"1,321","Biography, Drama, History","Liam Neeson, Ralph Fiennes, Ben Kingsley, Caro..."
5,Ojciec chrzestny II,The early life and career of Vito Corleone in ...,1974,1 October 1973,19 June 1974,Francis Ford Coppola,9.0,202,"1,235","Crime, Drama","Al Pacino, Robert De Niro, Robert Duvall, Dian..."
6,Dwunastu gniewnych ludzi,The jury in a New York City murder trial is fr...,1957,NaN,NaN,Sidney Lumet,9.0,96,"767,231","Crime, Drama","Henry Fonda, Lee J. Cobb, Martin Balsam, John ..."
7,Pulp Fiction,"The lives of two mob hitmen, a boxer, a gangst...",1994,20 September 1993,30 November 1993,Quentin Tarantino,8.9,154,"1,990","Crime, Drama","John Travolta, Uma Thurman, Samuel L. Jackson,..."
8,Incepcja,A thief who steals corporate secrets through t...,2010,19 June 2009,22 November 2009,Christopher Nolan,8.8,148,"2,278","Action, Adventure, Sci-Fi","Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio..."
9,Władca Pierścieni: Dwie wieże,While Frodo and Sam edge closer to Mordor with...,2002,11 October 1999,22 December 2000,Peter Jackson,8.8,179,"1,611","Action, Adventure, Drama","Elijah Wood, Ian McKellen, Viggo Mortensen, Or..."
